# Econ5150: Advanced Computing
Zhentao Shi

## Introduction

Modern econometric analysis is inseparable from computation. Estimation routines, simulation studies, and empirical policy evaluations all rely on translating mathematical models into efficient code. A naive translation might work for toy datasets, but production-scale projects routinely involve large datasets and tuning loops that magnify any inefficiency.


For research the goal is to produce code that is correct, fast enough, and maintainable.  
As an illustrative example, Lin, Shi, Wang, and Yan (2022)'s large-scale empirical projects consume 192 core-hours for a single estimation run, while it takes 8 wall-clock hours on 24 cores ($192 = 8 \times 24$). Such budgets make it infeasible to iterate manually. Thoughtful code architecture -- vectorized kernels, reusable functions, automation for hyperparameter sweeps are important.


Computational economics juggles two scarce resources:
**Human time** is spent designing algorithms, reading documentation, and debugging. Optimizing prematurely wastes this resource.
**Machine time** is spent executing code. Ignoring efficiency jeopardizes deadlines when facing large simulations.

The art is to prototype quickly, profile bottlenecks, and then invest effort only where performance matters.

## Vectorization

Vectorization replaces explicit Python loops with operations implemented in lower-level languages. NumPy, pandas, and SciPy explore these routines through array expressions, broadcasting rules, and linear algebra primitives.

 
Nowadays with vibe coding, AI is likely to follow the mathematical expressions, instead of reasoning by itself about the choices. 
Although vectorization is in general encouraged, if linear algebra is the bottleneck, careful testing will be essential for the performance of the code. 


### Case Study: Heteroskedasticity-Robust Variance
In OLS with heteroskedastic errors the asymptotic variance of the estimator is
$$\sqrt{n}(\hat\beta-\beta_0) \xrightarrow{d} \mathcal{N}\!\left(0,\; Q^{-1} \Omega Q^{-1}\right),$$
where $Q = \mathbb{E}[x_i x_i']$ and
$\Omega = \mathbb{E}[x_i e_i e_i x_i']$. Its sample analog has the following expressions are equivalent:
$$
\underset{\mathrm{opt1}}{\frac{1}{n}\sum_{i=1}^{n}x_{i}x_{i}'\widehat{e}_{i}^{2}}=\underset{\mathrm{opt2,3}}{\frac{1}{n}X'DX}=\underset{\mathrm{opt 4}}{\frac{1}{n}\left(X'D^{1/2}\right)\left(D^{1/2}X\right)}
$$
where $D$ is a diagonal matrix of $\left(\widehat{e}_{1}^{2},\widehat{e}_{2,}^{2},\ldots,\widehat{e}_{n}^{2}\right)$. 
It admits multiple computational strategies:
1. Loop over observations and accumulate $\hat e_i^2 x_i x_i'$ into a running matrix.
2. Form $D = \operatorname{diag}(\hat e^2)$ and compute $X^\top D X$ using dense multiplies.
3. Treat $D$ as sparse to avoid populating off-diagonal zeros.
4. Compute $X^\top D^{1/2}$ once and reuse it to form an outer product.

All formulas agree mathematically, yet their run times differ dramatically depending on the size and sparsity of $X$. The best choice is problem-specific, and profiling guides the decision.

In [ ]:
import numpy as np
import time

def hac_loop(X, resid):
    n, k = X.shape
    sigma = np.zeros((k, k))
    for i in range(n):
        xi = X[i]
        sigma += resid[i] ** 2 * np.outer(xi, xi)
    return sigma / n

def hac_vectorized(X, resid):
    weighted = resid[:, None] * X
    return weighted.T @ weighted / X.shape[0]

def hac_einsum(X, resid):
    weighted = resid[:, None] * X
    return np.einsum('ni,nj->ij', weighted, weighted) / X.shape[0]

rng = np.random.default_rng(20260215)
X = rng.normal(size=(50_000, 5))
resid = rng.normal(size=50_000)

start = time.perf_counter()
_ = hac_loop(X, resid)
loop_time = time.perf_counter() - start

start = time.perf_counter()
_ = hac_vectorized(X, resid)
vec_time = time.perf_counter() - start

start = time.perf_counter()
_ = hac_einsum(X, resid)
einsum_time = time.perf_counter() - start

print(f'loop version:      {loop_time:0.3f} seconds')
print(f'vectorized:        {vec_time:0.3f} seconds')
print(f'einsum/batched:    {einsum_time:0.3f} seconds')

The loop version is easiest to read but pays the price of Python-level iteration. The second implementation treats each residual-weighted regressor as a row of a matrix and relies on highly optimized BLAS routines. The `einsum` formulation makes the contraction explicit and often matches the vectorized implementation while being friendlier to sparse back ends.

When comparing implementations, inspect both run time and memory demands. A dense matrix formulation that materializes an $n \times n$ object is infeasible even if NumPy executes each individual kernel quickly.

## 3. Profiling Before Optimizing
Optimization without measurement is guesswork. Adopt a disciplined workflow:
1. **Reproducible baseline**: wrap computations in functions and write unit tests or assertions verifying correctness.
2. **Time the code**: start with `time.perf_counter` or IPython's `%%timeit` for rough estimates.
3. **Profile hotspots**: use `cProfile`, `line_profiler`, or `scalene` to identify the functions and lines that dominate execution time.
4. **Diagnose causes**: determine whether the bottleneck is algorithmic complexity, memory bandwidth, Python overhead, or I/O.
5. **Iterate deliberately**: apply a single optimization at a time and re-run tests to prevent regressions.

Notebook workflows benefit from storing profiling output (for example in CSV or Markdown tables) so that future readers can understand the performance rationale.

### Instrumentation Patterns
- Wrap expensive loops in helper functions so that profilers can attribute time precisely.
- Prefer `range` over `while` and local variables over repeated attribute lookups inside loops.
- Cache pure computations (memoization) only when the hit rate is high; otherwise the bookkeeping costs dominate.
- When randomness matters, use seeded generators (`np.random.default_rng`) to guarantee reproducibility across optimization attempts.

## 4. Memory and Data Layout
Compute time is irrelevant if the program exhausts memory. Guidelines:
- Choose suitable data types (`float32`, `int32`) when precision permits; smaller dtypes reduce bandwidth pressure.
- Use `np.ascontiguousarray` or `np.asarray` to ensure arrays are aligned for BLAS routines.
- Chunk large jobs: process data in blocks and aggregate partial results to keep peak memory bounded.
- Exploit sparse structures (`scipy.sparse.csr_matrix`) when matrices contain many zeros.
- Avoid unnecessary copies; in pandas, prefer vectorized operations over `apply`, and use `assign` or `eval` with `engine='numexpr'` for arithmetic.

### When Vectorization Is Not Enough
If profiling shows Python overhead still dominates, consider:
- **Numba**: JIT-compile numerical kernels with minimal code changes by decorating functions with `@njit`.
- **Cython** or **pybind11**: write performant extensions in C/C++ while retaining Python interfaces.
- **Specialized libraries**: for FFTs, convolutions, or optimization routines, leverage domain-tailored packages before reinventing algorithms.

The choice depends on how often the code will be reused and the development effort you can justify.

## 5. Parallel and Distributed Computing
Parallelism increases throughput by dividing work across cores or machines. Three models appear frequently in empirical research:
- **Shared-memory parallelism** (threads, multiprocessing) suits moderate-sized tasks on a single workstation.
- **Task-based schedulers** (joblib, Dask, Ray) coordinate batches of medium-sized jobs, ideal for bootstraps or hyper-parameter sweeps.
- **Distributed clusters** use multiple nodes connected over a network, managed by schedulers such as SLURM or PBS.

Parallel speedups are bounded by Amdahl's Law: the serial fraction caps the overall gain. Maximize the parallelizable portion by isolating computational kernels and minimizing synchronization.

### Choosing a Parallel Strategy
Ask the following questions before parallelizing:
1. **Is the workload embarrassingly parallel?** Independent simulations or cross-validation folds are easiest to scale.
2. **What data must workers access?** Broadcasting large matrices to each process can negate speedups; consider memory-mapped files or shared arrays.
3. **How will results be aggregated?** Design reducers that combine partial outputs without race conditions.
4. **Does the environment impose limits?** Desktop operating systems restrict process counts; clusters may require queue submissions with wall-time requests.

For reproducibility, log the number of workers, random seeds, and per-task run times.

In [ ]:
import numpy as np
from multiprocessing import Pool, cpu_count

def _partial_hac(args):
    X_chunk, resid_chunk = args
    weighted = resid_chunk[:, None] * X_chunk
    return weighted.T @ weighted

def hac_parallel(X, resid, n_jobs=None):
    if n_jobs is None:
        n_jobs = max(1, cpu_count() - 1)
    partitions = np.array_split(np.arange(X.shape[0]), n_jobs)
    with Pool(processes=n_jobs) as pool:
        pieces = pool.map(
            _partial_hac,
            [(X[idx], resid[idx]) for idx in partitions]
        )
    return sum(pieces) / X.shape[0]

if __name__ == '__main__':
    rng = np.random.default_rng(20260215)
    X = rng.normal(size=(200_000, 6))
    resid = rng.normal(size=200_000)
    sigma_hat = hac_parallel(X, resid, n_jobs=4)
    print('parallel estimate has shape', sigma_hat.shape)

The helper `_partial_hac` computes a partial cross-product; the main function partitions the index set, dispatches tasks to worker processes, and aggregates the results. The `if __name__ == '__main__':` guard is required on Windows and macOS to prevent infinite process spawning when the module is imported.

## 6. Working with Clusters and Cloud Resources
University clusters and commercial clouds provide elastic compute, but using them efficiently demands planning.
- **Login nodes** are for file transfers, editing scripts, and light debugging. Heavy computations belong to compute nodes accessed through a scheduler.
- **Job schedulers** (SLURM, PBS, LSF) require resource requests -- cores, memory, wall time. Under-requesting causes failures; over-requesting lengthens queue time.
- **Environment setup** is reproducible when scripted: load modules, activate Conda environments, or use container images specified in job scripts.
- **Monitoring** involves tools like `squeue`, `sacct`, or cloud dashboards to track progress and diagnose errors.

Automate routine steps with shell scripts so that rerunning an experiment becomes a single command.

### Example SLURM Script Skeleton
```bash
#!/bin/bash
#SBATCH --job-name=econ5150-sim
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=32G
#SBATCH --time=04:00:00
#SBATCH --output=logs/%x-%j.out

module load python/3.11
source activate econ5150

python run_simulation.py --replications 1000 --seed 20260215
```
Adjust the directives to match the scheduler on your system (for example, `qsub` for PBS). Always test the script with a short wall-time and few replications before launching large jobs.

### Remote Workflow Checklist
1. **Develop locally**: prototype on small data and ensure scripts accept command-line arguments for inputs, outputs, and random seeds.
2. **Package dependencies**: export `environment.yml` or `requirements.txt`, or build a Docker or Apptainer image.
3. **Transfer resources**: use `scp`, `rsync`, or Git to move code and data snapshots to the cluster.
4. **Submit jobs**: launch batch scripts or array jobs; record job IDs for tracking.
5. **Collect outputs**: store results in versioned directories, compress large logs, and pull summaries back to your workstation.

## 7. Reproducibility and Collaboration
Reproducible computation shortens the path from draft to publication.
- Maintain a project structure with clear separation between raw data, processed data, code, and figures.
- Use version control (`git`) to track both code and configuration.
- Capture software environments with Conda, `requirements.txt`, or container recipes. Record exact package versions in your paper's appendix.
- Automate pipelines with Makefiles, `nox`, or simple shell scripts so that a single command can rebuild key tables.
- Document random seeds and experiment metadata (start time, git commit, hardware) in output files.

### Recreating Python Environments
```bash
# create an environment from a specification file
conda env create -f environment.yml
conda activate econ5150

# record the environment after installing additional packages
conda env export --from-history > environment.yml
```
For containerized workflows, adapt these commands in a `Dockerfile` or `apptainer.def` so that collaborators can launch identical environments on any machine.

## 8. Key Takeaways
- Start with clear mathematical expressions; translate them into vectorized code that mirrors the algebra.
- Measure performance before optimizing, and optimize only the true bottlenecks.
- Treat memory as a first-class constraint and design data flows that scale gracefully.
- Parallel computation is powerful when the workload is partitionable and aggregation is cheap.
- Reproducible workflows -- environment management, logging, automation -- turn computational experiments into reliable scientific evidence.

### Further Reading
- McKinney (2022), *Python for Data Analysis*, chapters on performance and vectorization.
- van der Walt, Colbert, and Varoquaux (2011), *The NumPy Array: A Structure for Efficient Numerical Computation*.
- Gorelick and Ozsvald (2020), *High Performance Python*, for profiling and parallel patterns.
- Documentation for your cluster's scheduler and the CUHK Econ computing resources portal.